# 第6章 Gated RNN

RNN 在许多情况下它都无法很好地学习到时序数据的长期依赖关系。

简单 RNN 经常被名为 LSTM 或 GRU 的层所代替。
实际上，当我们说 RNN 时，更多的是指 LSTM 层。

LSTM 和 GRU 中增加了一种名为“门”的结构。基于这个门，可以学
习到时序数据的长期依赖关系。

## 6.1 RNN的问题

RNN 之所以不擅长学习时序数据的长期依赖关系，是因为 BPTT 会发生梯度消失和梯度爆炸的问题。

### 6.1.1 RNN的复习

![](../images/图6-1.RNN层：循环展开前和展开后.PNG)
图6-1.RNN层：循环展开前和展开后

在图 6-1 中，当输入时序数据 $x_t$ 时，RNN 层输出 $h_t$。这个 $h_ t$ 也称为
RNN 层的隐藏状态，它记录过去的信息。
RNN 的特点在于使用了上一时刻的隐藏状态，由此，RNN 可以继承过
去的信息。

![](../images/图6-2.RNN层的计算图（MatMul节点表示矩阵乘积）.PNG)
图6-2.RNN层的计算图（MatMul节点表示矩阵乘积）

### 6.1.2 梯度消失和梯度爆炸

![](../images/图6-3.某种程度上需要“长期记忆”的问题示例..PNG)
图6-3.某种程度上需要“长期记忆”的问题示例：“?”中应填入什么单词?

![](../images/图6-4.正确解标签为Tom时梯度的流动.PNG)
图6-4.正确解标签为Tom时梯度的流动

在学习正确解标签 Tom 时，重要的是 RNN 层的存在。RNN 层通过
向过去传递“有意义的梯度”，能够学习时间方向上的依赖关系。此时梯度
（理论上）包含了那些应该学到的有意义的信息，通过将这些信息向过去传
递，RNN 层学习长期的依赖关系。但是，如果这个梯度在中途变弱（甚至
没有包含任何信息），则权重参数将不会被更新。也就是说，RNN 层无法学
习长期的依赖关系。不幸的是，随着时间的回溯，这个简单 RNN 未能避免
梯度变小（梯度消失）或者梯度变大（梯度爆炸）的命运。

### 6.1.3 梯度消失和梯度爆炸的原因

![](../images/图6-5.RNN层在时间方向上的梯度传播.PNG)
图6-5.RNN层在时间方向上的梯度传播

![](../images/图6-6.y%20=%20tanh(x)的图（虚线是导数）.PNG)
图6-6.y = tanh(x)的图（虚线是导数）

如果经过 tanh 函数 T 次，则梯度也会减小 T 次。

![](../images/图6-7.仅关注RNN层的矩阵乘积时的反向传播的梯度.PNG)
图6-7.仅关注RNN层的矩阵乘积时的反向传播的梯度

这里需要注意的是，每一次矩阵乘积计算都使用相同的权重 $W_h$。

反向传播时梯度的值通过 MatMul 节点时会如何变化呢？
观察梯度大小的变化（[ch06/rnn_gradient_graph.py](../ch06/rnn_gradient_graph.py)）

![](../images/图6-8.梯度dh的大小随时间步长呈指数级增加.PNG)
图6-8.梯度dh的大小随时间步长呈指数级增加

梯度的大小随时间步长呈指数级增加，这就是**梯度爆炸**（exploding gradients）。
如果发生梯度爆炸，最终就会导致溢出，出现 NaN（Not a Number，非数值）之类的值。
如此一来，神经网络的学习将无法正确运行。

![](../images/图6-9.梯度dh的大小随时间步长呈指数级减小.PNG)
图6-9　梯度dh的大小随时间步长呈指数级减小

梯度呈指数级减小，这就是**梯度消失**（vanishing gradients）。如果发生梯度消失，梯度将迅速变小。
一旦梯度变小，权重梯度不能被更新，模型就会无法学习长期的依赖关系。


如果 $W_h$ 不是标量，而是矩阵呢？此时，矩阵的奇异值将成为指标。
简单而言，矩阵的奇异值表示数据的离散程度。根据这个奇异值（更准
确地说是多个奇异值中的最大值）是否大于 1，可以预测梯度大小的变化。

如果奇异值的最大值大于 1，则可以预测梯度很有可能会呈指数
级增加；而如果奇异值的最大值小于 1，则可以判断梯度会呈指
数级减小。但是，并不是说奇异值比 1 大就一定会出现梯度爆炸。
也就是说，这是必要条件，并非充分条件。

### 6.1.4 梯度爆炸的对策

解决梯度爆炸有既定的方法，称为**梯度裁剪**（gradients clipping）。

$$
if  \left\| \hat^{g} \right\| \geqslant threshold: \\
    \hat{g} = \frac{threshold}{\left\| \hat^{g} \right\| } \hat{g}
$$

这里假设可以将神经网络用到的所有参数的梯度整合成一个，并用符号 $\hat{g}$ 表
示。另外，将阈值设置为 threshold。此时，如果梯度的 L2 范数 $\left \|  \hat{g} \right \|$ 大于或
等于阈值，就按上述方法修正梯度，这就是梯度裁剪。

$\hat{g}$ 整合了神经网络中用到的所有参数的梯度。比如，当某个模型
有 W1和 W2两个参数时，$\hat{g}$ 就是这两个参数对应的梯度 dW1和 dW2
的组合。

我们用 Python 来实现梯度裁剪，将其实现为 clip_grads(grads, max_norm) 函数。
参数 grads 是梯度的列表，max_norm 是阈值，此时梯度裁剪的实现（[ch06/clip_grads.py](../ch06/clip_grads.py)）

## 6.2 梯度消失和LSTM

为了解决在 RNN 的学习中，梯度消失的问题。需要从根本上改变 RNN 层的结构，这里本章的主题 Gated RNN 就要登场了。

Gated RNN 框架（网络结构），其中具有代表性的有LSTM 和 GRU。

### 6.2.1 LSTM接口

![](../images/图6-10.应用了简略图示法的RNN层：本节使用简略图示法以方便观察.PNG)
图6-10.应用了简略图示法的RNN层：本节使用简略图示法以方便观察

![](../images/图6-11.RNN层与LSTM层的比较.PNG)
图6-11. RNN层与LSTM层的比较

LSTM 与 RNN 的接口的不同之处在于，LSTM 还有路径 c。这个 c 称为记忆单元（或者简称为“单元”），相当于 LSTM 专用
的记忆部门。

记忆单元在 LSTM 层内部结束工作，不向其他层输出。而 LSTM 的隐藏状
态 h 和 RNN 层相同，会被（向上）输出到其他层。

### 6.2.2 LSTM层的结构

[colah’s blog: Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
[colah’s blog: Understanding LSTM Networks-翻译](https://zhuanlan.zhihu.com/p/84333843)

![](../images/图6-12.LSTM层基于记忆单元ct计算隐藏状态ht.PNG)
图6-12.LSTM层基于记忆单元ct计算隐藏状态ht

如图 6-12 所示，当前的记忆单元 $c_t$ 是基于 3 个输入 $c_{t−1}$、$h_{t−1}$ 和 $x_t$，
经过“某种计算”（后述）算出来的。这里的重点是隐藏状态 $h_t$ 要使用更新
后的 $c_t$ 来计算。另外，这个计算是 $h_t = \tanh(c_t)$，表示对 $c_t$ 的各个元素应
用 tanh 函数。

Gate 的功能：Gate 是“门”的意思，就像将门打开或合上一样，控制数据的流动。

![](../images/图6-13.门的比喻：控制水流.PNG)
图6-13.门的比喻：控制水流

![](../images/图6-14.将水的流量控制在0.0-1.0的范围内.PNG)
图6-14.将水的流量控制在0.0 ~ 1.0的范围内

如图 6-14 所示，门的开合程度由 0.0 ~1.0 的实数表示（1.0 为全开），
通过这个数值控制流出的水量。这里的重点是，门的开合程度也是（自动）
从数据中学习到的。

### 6.2.3 输出门

针对 $\tanh(c_t)$ 的各个元素，调整它们作为下一时刻的隐藏状态的重要程度。
由于这个门管理下一个隐藏状态 $h_t$ 的输出，所以称为**输出门**（output gate）

sigmoid 函数用 σ( ) 表示。

$$
    o = \sigma(x_t W_x^{(o)} + h_{t-1}W_h^{(o)}  + b^{(o)}) \tag{6.1}
$$

![](../images/图6-15.添加输出门.PNG)
图6-15.添加输出门

将输出门进行的式 (6.1) 的计算表示为 $\sigma$。然后，将它的
输出表示为 $o$，则 $h_t$ 可由 $o$ 和 $tanh(c_t)$ 的乘积计算出来。这里说的“乘积”
是对应元素的乘积，也称为阿达玛乘积。如果用 $\odot$ 表示阿达玛乘积，则此
处的计算如下所示：

$$
    h_t = o \odot \tanh(c_t) \tag{6.2}
$$

以上就是 LSTM 的输出门。

tanh的输出是−1.0 ~ 1.0的实数。我们可以认为这个−1.0 ~ 1.0的
数值表示某种被编码的“信息”的强弱（程度）。而sigmoid 函数的
输出是0.0~1.0的实数，表示数据流出的比例。因此，在大多数情
况下，门使用sigmoid函数作为激活函数，而包含实质信息的数据
则使用tanh函数作为激活函数。

### 6.2.4 遗忘门

在记忆单元 $c_{t−1}$ 上添加一个忘记不必要记忆的门，这里称为**遗忘门**（forget gate）。

![](../images/图6-16.添加遗忘门.PNG)
图6-16.添加遗忘门

$$
    f = \sigma(x_t W_x^{(f)} + h_{t-1}W_h^{(f)}  + b^{(f)}) \tag{6.3}
$$

$c_t$ 由这个 $f$ 和上一个记忆单元 $c_{t−1}$ 的对应元素的乘积求得（$c_t = f \odot c_{t−1}$）。

### 6.2.5 新的记忆单元

遗忘门从上一时刻的记忆单元中删除了应该忘记的东西，但是这样一
来，记忆单元只会忘记信息。现在我们还想向这个记忆单元添加一些应当记
住的新信息，为此我们添加新的 tanh 节点

![](../images/图6-17.向新的记忆单元添加必要信息.PNG)
图6-17.向新的记忆单元添加必要信息

$$
     g = \tanh(x_t W_x^{(g)} + h_{t-1}W_h^{(g)}  + b^{(g)}) \tag{6.4}
$$

$g$ 表示向记忆单元添加的新信息。通过将这个 $g$ 加到上一时刻的 $c_{t−1}$上，从而形成新的记忆。

### 6.2.6 输入门

给 $g$ 添加门，这里将这个新添加的门称为**输入门**（input gate）。

![](../images/图6-18.添加输入门.PNG)
图6-18.添加输入门

$$
     i = \sigma(x_t W_x^{(i)} + h_{t-1}W_h^{(i)}  + b^{(i)}) \tag{6.5}
$$

将 $i$ 和 $g$ 的对应元素的乘积添加到记忆单元中。

### 6.2.7 LSTM的梯度的流动

![](../images/图6-19.记忆单元的反向传播.PNG)
图6-19.记忆单元的反向传播

在之前的 RNN 的反向传播中，我们使用相同的权重矩
阵重复了多次矩阵乘积计算，由此导致了梯度消失（或梯度爆炸）。而这里
的 LSTM 的反向传播进行的不是矩阵乘积计算，而是对应元素的乘积计算，
而且每次都会基于不同的门值进行对应元素的乘积计算。这就是它不会发生
梯度消失（或梯度爆炸）的原因。

遗忘门认为“应该忘记”的记忆单元的元素，其梯度会变小；而遗忘门认为“不能
忘记”的元素，其梯度在向过去的方向流动时不会退化。因此，可以期待记忆
单元的梯度（应该长期记住的信息）能在不发生梯度消失的情况下传播。

LSTM的记忆单元不会（难以）发生梯度消失。因此，
可以期待记忆单元能够保存（学习）长期的依赖关系。

LSTM是Long Short-Term Memory（长短期记忆）的缩写，意思是
可以长（Long）时间维持**短期记忆**（Short-Term Memory）。

## 6.3 LSTM的实现

LSTM 中进行的计算：
$$
    f = \sigma(x_t W_x^{(f)} + h_{t-1}W_h^{(f)}  + b^{(f)})     \\
    g = \tanh(x_t W_x^{(g)} + h_{t-1}W_h^{(g)}  + b^{(g)})     \\
    i = \sigma(x_t W_x^{(i)} + h_{t-1}W_h^{(i)}  + b^{(i)})     \\
    o = \sigma(x_t W_x^{(o)} + h_{t-1}W_h^{(o)}  + b^{(o)})     \\
    \tag{6.6}
$$

$$
    c_t = f \odot c_{t-1} + g \odot i   \tag{6.7}
$$

$$
    h_t = o \odot \tanh(c_t)    \tag{6.8}
$$

以上就是 LSTM 进行的计算。这里需要注意式 (6.6) 中的 4 个仿射变换。
这里的仿射变换是指 $xW_x + hW_h + b$ 这样的式子。式 (6.6) 中通过 4 个式
子分别进行仿射变换，但其实可以整合为通过 1 个式子进行，如图 6-20 所示。

![](../images/图6-20.整合4个权重，通过1次仿射变换进行4个计算.PNG)
图6-20.整合4个权重，通过1次仿射变换进行4个计算

矩阵库计算“大矩阵”时通常会更快，而且通过将权重整合到一起管理，源代码也会更简洁。

![](../images/图6-21.整合4个权重进行仿射变换的LSTM的计算图.PNG)
图6-21.整合4个权重进行仿射变换的LSTM的计算图

![](../images/图6-22射变换的形状的改变（省略偏置）.PNG)
图6-22.仿射变换的形状的改变（省略偏置）

![](../images/图6-23.slice节点的正向传播（上）和反向传播（下）.PNG)
图6-23.slice节点的正向传播（上）和反向传播（下）

### Time LSTM层的实现

Time LSTM 层是整体处理 T 个时序数据的层，由 T 个 LSTM 层构成：

![](../images/图6-24.TimeLSTM的输入和输出.PNG)
图6-24.Time LSTM的输入和输出

![](../images/图6-25.TimeLSTM的反向传播的输入和输出.PNG)
图6-25.Time LSTM的反向传播的输入和输出

TimeLSTM 实现（[common/time_layers.py](../common/time_layers.py)）


## 6.4 使用LSTM的语言模型

![](../images/图6-26.语言模型的网络结构。左图是上一章创建的使用Time%20RNN的模型，右图是本.PNG)
图6-26.语言模型的网络结构。左图是上一章创建的使用Time RNN的模型，右图是本章创建的使用Time LSTM的模型

（[ch06/rnnlm.py](../ch06/rnnlm.py)）
在 PTB 数据集上学习这个网络：（[ch06/train_rnnlm.py](../ch06/train_rnnlm.py)）

![](../images/图6-27.终端的输出结果.PNG)
图6-27.终端的输出结果

![](../images/图5-28.将Time_Affine层实现为T个Affine层的集合.PNG)
图6-28.困惑度的演变（每20次迭代对训练数据进行1次评价）

## 6.5 进一步改进RNNLM

### 6.5.1 LSTM层的多层化

![](../images/图6-29.使用两个LSTM层的RNNLM.PNG)
图6-29.使用两个LSTM层的RNNLM

在 PTB 数据集上学习语言模型的情况下，当 LSTM 的层数为 2 ～ 4 时，可以获得比较好的结果。


### 6.5.2 基于Dropout抑制过拟合

通过加深层，可以创建表现力更强的模型，但是这样的模型往往会发生**过拟合**（overfitting）。
RNN 比常规的前馈神经网络更容易发生过拟合，因此 RNN 的过拟合对策非常重要。

过拟合是指过度学习了训练数据的状态，也就是说，过拟合是一种
缺乏泛化能力的状态。我们想要的是一个泛化能力强的模型，因此
必须基于训练数据和验证数据的评价差异，判断是否发生了过拟合，
并据此来进行模型的设计。


抑制过拟合已有既定的方法：
一是增加训练数据；
二是降低模型的复杂度。我们会优先考虑这两个方法。
除此之外，对模型复杂度给予惩罚的正则化也很有效。

![](../images/图6-30.Dropout的概念图.PNG)
图6-30.Dropout的概念图

![](../images/图6-31.将Dropout层应用于前馈神经网络的例子.PNG)
图6-31.将Dropout层应用于前馈神经网络的例子

![](../images/图6-32.不好的例子：在时序方向上插入Dropout层.PNG)
图6-32.不好的例子：在时序方向上插入Dropout层

如果在时序方向上插入 Dropout，那么当模型学习时，随着时间的推移，信息会渐渐丢失。
也就是说，因 Dropout 产生的噪声会随时间成比例地积累。考虑到噪声的积累，最好不要在时间轴方向上插入 Dropout。
因此，如图 6-33 所示，我们在深度方向（垂直方向）上插入 Dropout 层。
![](../images/图6-33.好的例子：在深度方向（垂直方向）上插入Dropout层.PNG)
图6-33.好的例子：在深度方向（垂直方向）上插入Dropout层

这样一来，无论沿时间方向（水平方向）前进多少，信息都不会丢失。
Dropout 与时间轴独立，仅在深度方向（垂直方向）上起作用。

“常规的 Dropout”不适合用在时间方向上。但是，最近的
研究提出了多种方法来实现时间方向上的 RNN 正则化。
“变分 Dropout”（variational dropout）就被成功地应用在了时间
方向上。
除了深度方向，变分 Dropout 也能用在时间方向上，从而进一步提高
语言模型的精度。如图 6-34 所示，它的机制是同一层的 Dropout 使用相同
的 mask。这里所说的 mask 是指决定是否传递数据的随机布尔值。

![](../images/图6-34.变分Dropout的例子.PNG)
图6-34.变分Dropout的例子

如图 6-34 所示，通过同一层的 Dropout 共用 mask，mask 被“固定”。
如此一来，信息的损失方式也被“固定”，所以可以避免常规 Dropout 发生
的指数级信息损失。

### 6.5.3 权重共享

改进语言模型有一个非常简单的技巧，那就是**权重共享**（weight tying），weight tying 可以直译为“权重绑定”，其含义就是共享权重。

![](../images/图6-35.语言模型中共享权重的例子：Embedding层和Softmax前的Affine层共享权重.PNG)
图6-35.语言模型中共享权重的例子：Embedding层和Softmax前的Affine层共享权重

为什么说权重共享是有效的呢？直观上，共享权重可以减少需要学
习的参数数量，从而促进学习。另外，参数数量减少，还能收获抑
制过拟合的好处。

### 6.5.4 更好的RNNLM的实现

![](../images/图6-36.BetterRnnlm类的网络结构.PNG)
图6-36.BetterRnnlm类的网络结构

改进的 3 点如下：
* LSTM 层的多层化（此处为 2 层）
* 使用 Dropout（仅应用在深度方向上）
* 权重共享（Embedding 层和 Affine 层的权重共享）

### 6.5.5 前沿研究

![](../images/图6-37.各模型在PTB数据集上的结果.PNG)
图6-37.各模型在PTB数据集上的结果

## 6.6 小结

## 本章所学的内容

* 在简单 RNN 的学习中，存在梯度消失和梯度爆炸问题
* 梯度裁剪对解决梯度爆炸有效，LSTM、GRU 等 Gated RNN 对解决梯度消失有效
* LSTM 中有 3 个门：输入门、遗忘门和输出门
* 门有专门的权重，并使用 sigmoid 函数输出 0.0 ～ 1.0 的实数
* LSTM 的多层化、Dropout 和权重共享等技巧可以有效改进语言模型
* RNN 的正则化很重要，人们提出了各种基于 Dropout 的方法